In [1]:
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

# 파일 저장을 위한 라이브러리
from pathlib import Path

# video statistic(좋조회수/좋아요/쇼츠,롱폼 여부) 함수
from videos import (
    fetch_video_statistics_batch,
    chunked,
)

In [2]:
BASE_DIR = Path("..")        # src 기준
DATA_DIR = BASE_DIR / "data"
INPUT_PATH = DATA_DIR / "channel_all_videos.csv"

df_videos = pd.read_csv(INPUT_PATH)

df_videos.head(), df_videos.shape


(  channel_label   channel_handle     video_id  \
 0   ISEGYE_IDOL  isegyeidol_ofcl  YkNTjfc45-o   
 1   ISEGYE_IDOL  isegyeidol_ofcl  _KpApEqZp9c   
 2   ISEGYE_IDOL  isegyeidol_ofcl  bZHiZn5ea90   
 3   ISEGYE_IDOL  isegyeidol_ofcl  im_YVykT91E   
 4   ISEGYE_IDOL  isegyeidol_ofcl  -xKhVTH9z3s   
 
                                          video_title          published_at  
 0  (SUB) 이게 나온다고요?! 😂🎁 랜덤 선물 뽑기 - 화이트 (White) 발매 ...  2026-01-09T09:00:56Z  
 1          이심전심 이세계아이돌 💞 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:29Z  
 2  저음불가 르르땅 🦊 과 과몰입 징부기 🍔 #이세계아이돌 #ISEGYEIDOL #sh...  2026-01-06T09:00:25Z  
 3   나만 아니면 돼애애애애!!!!!! 🤭 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:19Z  
 4  (SUB) 게임에 진심인데요...!🔥 운명의 트리 꾸미기 🎄 - 화이트 (White...  2026-01-06T09:00:01Z  ,
 (5157, 5))

In [3]:
video_ids = df_videos["video_id"].dropna().unique().tolist()

len(video_ids)

5157

In [4]:
video_stats_map = {}

for chunk in chunked(video_ids, size=50):
    batch_stats = fetch_video_statistics_batch(chunk)
    video_stats_map.update(batch_stats)

len(video_stats_map)

5157

In [5]:
stats_df = (
    pd.DataFrame
    .from_dict(video_stats_map, orient="index")
    .reset_index()
    .rename(columns={"index": "video_id"})
)

stats_df.head()

,video_id,view_count,like_count,comment_count,duration_sec,video_type
0,YkNTjfc45-o,26203,2897,151,641.0,LONG
1,_KpApEqZp9c,125626,6939,140,48.0,SHORT
2,bZHiZn5ea90,56493,4480,110,57.0,SHORT
3,im_YVykT91E,59328,4560,127,59.0,SHORT
4,-xKhVTH9z3s,32345,3382,213,732.0,LONG


In [6]:
df_videos_full = df_videos.merge(
    stats_df,
    on="video_id",
    how="left"
)

df_videos_full.head(), df_videos_full.shape

(  channel_label   channel_handle     video_id  \
 0   ISEGYE_IDOL  isegyeidol_ofcl  YkNTjfc45-o   
 1   ISEGYE_IDOL  isegyeidol_ofcl  _KpApEqZp9c   
 2   ISEGYE_IDOL  isegyeidol_ofcl  bZHiZn5ea90   
 3   ISEGYE_IDOL  isegyeidol_ofcl  im_YVykT91E   
 4   ISEGYE_IDOL  isegyeidol_ofcl  -xKhVTH9z3s   
 
                                          video_title          published_at  \
 0  (SUB) 이게 나온다고요?! 😂🎁 랜덤 선물 뽑기 - 화이트 (White) 발매 ...  2026-01-09T09:00:56Z   
 1          이심전심 이세계아이돌 💞 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:29Z   
 2  저음불가 르르땅 🦊 과 과몰입 징부기 🍔 #이세계아이돌 #ISEGYEIDOL #sh...  2026-01-06T09:00:25Z   
 3   나만 아니면 돼애애애애!!!!!! 🤭 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:19Z   
 4  (SUB) 게임에 진심인데요...!🔥 운명의 트리 꾸미기 🎄 - 화이트 (White...  2026-01-06T09:00:01Z   
 
    view_count  like_count  comment_count  duration_sec video_type  
 0       26203        2897            151         641.0       LONG  
 1      125626        6939            140          48.0      SHORT  
 2       56493 

In [ ]:
# Unknown 1개 실시간 스트리밍 영상
df_videos_full["video_type"].value_counts()

video_type
SHORT      3259
LONG       1897
UNKNOWN       1
Name: count, dtype: int64

In [ ]:
BASE_DIR = Path("..")        # src 기준
DATA_DIR = BASE_DIR / "data"
OUTPUT_PATH = DATA_DIR / "channel_videos_with_stats.csv"

df_videos_full.to_csv(OUTPUT_PATH, index=False)

print(f"Saved to {OUTPUT_PATH}")